## **Airports**

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome()
driver.get("https://www.ind.com/flights")

time.sleep(5) # Waiting unitl the data is loaded

soup = BeautifulSoup(driver.page_source, "html.parser")

value = []
table = soup.select_one("table.flight-destinations__table")
for row in table.select("tbody"):
    cols = row.find_all("td")
    row_values = []
    for col in cols:
        text = col.text.strip()        
        combined = text
        row_values.append(combined)

    value.append(row_values)

value = value[0]
driver.quit()

In [2]:
import pandas as pd
df = pd.DataFrame(value, columns=['AirportName'])
df['AirportCode'] = df['AirportName'].apply(lambda x: x.split('(')[-1].split(')')[0])
df['AirportName'] = df['AirportName'].apply(lambda x: x.split('*')[0].split('(')[0])
df.loc[len(df)] = ['Indianapolis', 'IND']
df_airports = df
df_airports

,AirportName,AirportCode
0,Atlanta,ATL
1,Austin-Bergstrom,AUS
2,Baltimore,BWI
3,Boston,BOS
4,Cancun,CUN
5,Charleston,CHS
6,Charlotte,CLT
7,Chicago-Midway,MDW
8,Chicago O'Hare,ORD
9,Dallas-Fort Worth,DFW


In [3]:
import sqlite3

conn = sqlite3.connect('FIDS.db')
cursor = conn.cursor()

sql = """
INSERT OR IGNORE INTO Airports (AirportCode, AirportName)
VALUES (?, ?)
"""

values = df_airports[['AirportCode', 'AirportName']].values.tolist()

cursor.executemany(sql, values)

conn.commit()

conn.close()

## **Airlines**

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome()
driver.get("https://www.ind.com/flights")

time.sleep(5) # Waiting unitl the data is loaded

soup = BeautifulSoup(driver.page_source, "html.parser")

value = []
table = soup.select_one("table.flight-airlines__table")
for row in table.select("tbody"):
    cols = row.find_all("td")
    row_values = []
    for col in cols:
        text = col.text.strip()        
        combined = text
        row_values.append(combined)

    value.append(row_values)

value = value[0]
value = value[0::3]

driver.quit()

In [5]:
df = pd.DataFrame(value, columns=['AirlineName'])
AirlineCode = pd.Series(['AC', 'AS', 'G4', 'AA', 'DL', 'F9', 'WN', 'NK','SY', 'UA'])
df['AirlineCode'] = AirlineCode
df_airlines = df
df_airlines

,AirlineName,AirlineCode
0,Aer Lingus,AC
1,Air Canada,AS
2,Alaska Air,G4
3,Allegiant,AA
4,American,DL
5,Delta,F9
6,Frontier,WN
7,Southwest,NK
8,Spirit,SY
9,Sun Country,UA


In [6]:
import sqlite3


conn = sqlite3.connect('FIDS.db')
cursor = conn.cursor()


sql = """
INSERT OR IGNORE INTO Airlines (AirlineCode, AirlineName)
VALUES (?, ?)
"""

values = df_airlines[['AirlineCode', 'AirlineName']].values.tolist()


cursor.executemany(sql, values)


conn.commit()
conn.close()

## **Remarks**

In [7]:
import sqlite3

conn = sqlite3.connect('FIDS.db') 
cursor = conn.cursor()

sql = "INSERT or ignore INTO Remarks (RemarkCode, RemarkName, UseYn) VALUES (?, ?, ?)"

remark_values = [
    ('PRG', 'Progressing', 'Y'),
    ('BRD', 'Boarding', 'Y'),
    ('DPT', 'Departed', 'Y'),
    ('ARR', 'Arrived', 'Y'),
    ('DLT', 'Delayed', 'Y'),
    ('CNL', 'Cancelled', 'Y'),
    ('ONT', 'On Time', 'Y'),
    ('ERL', 'Early', 'Y')
]

cursor.executemany(sql, remark_values)
conn.commit()
conn.close()

## **Users, Roles, UserRoles**

In [8]:
# conn = pymysql.connect(
#     host='localhost',
#     user='root',
#     password='root',
#     db='FIDS',
#     port=8889,
#     charset='utf8mb4'
# )
# cursor = conn.cursor()

# sql_roles = """
# INSERT INTO Roles (RoleID, RoleName) VALUES
# ('ADMIN', 'Administrator'),
# ('OPS', 'Operations Manager'),
# ('VIEWER', 'Viewer');
# """

# sql_users = """
# INSERT INTO Users (UserID, UserName, Password, AirportCode, AirlineCode) VALUES
# ('user01', 'Alice Johnson', '5e884898da28047151d0e56f8dc62927', 'ATL', 'AA'),
# ('user02', 'Bob Smith', '6cb75f652a9b52798eb6cf2201057c73', 'IND', 'DL'),
# ('user03', 'Charlie Kim', '2b3a8d0a34e3e237c54b4cf2d80a3c41', 'IND', 'AA');
# """

# sql_user_roles = """
# INSERT INTO UserRoles (UserRoleID, UserID, RoleID) VALUES
# ('UR01', 'user01', 'ADMIN'),
# ('UR02', 'user02', 'OPS'),
# ('UR03', 'user03', 'VIEWER');
# """

# cursor.execute(sql_roles)
# cursor.execute(sql_users)
# cursor.execute(sql_user_roles)
# conn.commit()
# conn.close()

## **ActiveFlightSchedules**

In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome()
driver.get("https://www.ind.com/flights/flight-status/arrivals?flightNumber=&airline=all&arrdep=A")

time.sleep(5) # Waiting unitl the data is loaded

soup = BeautifulSoup(driver.page_source, "html.parser")

theader = soup.select_one("thead.flight-table__header")

head = []
for row in theader.select("tr"):
    cols = row.find_all("th")
    head = [col.text.strip() for col in cols]

value = []
tbody = soup.select_one("tbody.flight-table__body")
for row in tbody.select("tr.flight-table__row"):
    cols = row.find_all("td")

    row_values = []

    for col in cols:
        text = col.text.strip()
        img = col.find("img")
        if img:
            img_src = 'https://www.ind.com'+img.get("src")
            combined = f"{text} ({img_src})" if text else img_src
        else:
            combined = text
        row_values.append(combined)

    value.append(row_values)

driver.quit()

In [10]:
import pandas as pd
df_a = pd.DataFrame(value, columns=head)
df_a

,Airline,Flight #,Origin,ETA,Gate #,Status
0,https://www.ind.com/assets/images/airlines/dl.png,DL 4909,Detroit,12:16 am,A8,Arrived
1,https://www.ind.com/assets/images/airlines/wn.png,WN 1373,Denver,12:06 am,B25,Arrived
2,https://www.ind.com/assets/images/airlines/aa.png,AA 2919,Chicago/O'Hare,12:17 am,,Arrived
3,https://www.ind.com/assets/images/airlines/dl.png,DL 3153,Atlanta,12:05 am,A5,Arrived
4,https://www.ind.com/assets/images/airlines/dl.png,DL 2233,Minneapolis-St. Paul,12:29 am,A13,Arrived
...,...,...,...,...,...,...
150,https://www.ind.com/assets/images/airlines/ua.png,UA 2281,Houston,11:55 pm,,On Time
151,https://www.ind.com/assets/images/airlines/ua.png,UA 6021,Wash. Dulles,11:56 pm,,On Time
152,https://www.ind.com/assets/images/airlines/aa.png,AA 2518,Miami,11:56 pm,,On Time
153,https://www.ind.com/assets/images/airlines/dl.png,DL 4909,Detroit,11:44 pm,A8,Early


In [11]:
# Preprocessing
from datetime import datetime
import numpy as np

status_mapping = {
    'Progressing': 'PRG',
    'Boarding': 'BRD',
    'Departed': 'DPT',
    'Arrived': 'ARR',
    'Delayed': 'DLT',
    'Cancelled': 'CNL',
    'On Time': 'ONT',
    'Early': 'ERL'
}

df_a['FlightNumber'] = df_a['Flight #'].apply(lambda x: x.split(' ')[1])
df_a['AirlineCode'] = df_a['Flight #'].apply(lambda x: x.split(' ')[0])
df_a['AirportCode'] = 'IND' # Indianapolis
df_a['ScheduledDate'] = datetime.today().strftime('%Y%m%d')
df_a['ScheduledTime'] = df_a['ETA'].apply(
    lambda x: datetime.strptime(x.strip().lower(), "%I:%M %p").strftime("%H%M"))
df_a['EstimatedDate'] = datetime.today().strftime('%Y%m%d')
df_a['EstimatedTime'] = df_a['ETA'].apply(
    lambda x: datetime.strptime(x.strip().lower(), "%I:%M %p").strftime("%H%M"))
df_a['OriginDestAirport'] = np.random.choice(df_airports['AirportCode'].dropna().unique(), size=len(df_a)) # Arbitrary data as airports data is not perfectly matching with those of df

#df['CheckinCounter'] = 
df_a['BoardingGate'] = df_a['Gate #']
#df['BaggageClaimBelt'] = 
df_a['Remarks'] = df_a['Status'].map(status_mapping)

df_a['FlightType'] = 'A'

df_a

,Airline,Flight #,Origin,ETA,Gate #,Status,FlightNumber,AirlineCode,AirportCode,ScheduledDate,ScheduledTime,EstimatedDate,EstimatedTime,OriginDestAirport,BoardingGate,Remarks,FlightType
0,https://www.ind.com/assets/images/airlines/dl.png,DL 4909,Detroit,12:16 am,A8,Arrived,4909,DL,IND,20250502,0016,20250502,0016,PUJ,A8,ARR,A
1,https://www.ind.com/assets/images/airlines/wn.png,WN 1373,Denver,12:06 am,B25,Arrived,1373,WN,IND,20250502,0006,20250502,0006,PHX,B25,ARR,A
2,https://www.ind.com/assets/images/airlines/aa.png,AA 2919,Chicago/O'Hare,12:17 am,,Arrived,2919,AA,IND,20250502,0017,20250502,0017,SFB,,ARR,A
3,https://www.ind.com/assets/images/airlines/dl.png,DL 3153,Atlanta,12:05 am,A5,Arrived,3153,DL,IND,20250502,0005,20250502,0005,DTW,A5,ARR,A
4,https://www.ind.com/assets/images/airlines/dl.png,DL 2233,Minneapolis-St. Paul,12:29 am,A13,Arrived,2233,DL,IND,20250502,0029,20250502,0029,BOS,A13,ARR,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,https://www.ind.com/assets/images/airlines/ua.png,UA 2281,Houston,11:55 pm,,On Time,2281,UA,IND,20250502,2355,20250502,2355,PDX,,ONT,A
151,https://www.ind.com/assets/images/airlines/ua.png,UA 6021,Wash. Dulles,11:56 pm,,On Time,6021,UA,IND,20250502,2356,20250502,2356,AUS,,ONT,A
152,https://www.ind.com/assets/images/airlines/aa.png,AA 2518,Miami,11:56 pm,,On Time,2518,AA,IND,20250502,2356,20250502,2356,PGD,,ONT,A
153,https://www.ind.com/assets/images/airlines/dl.png,DL 4909,Detroit,11:44 pm,A8,Early,4909,DL,IND,20250502,2344,20250502,2344,MYR,A8,ERL,A


In [12]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome()
driver.get("https://www.ind.com/flights/flight-status/departures?flightNumber=&airline=all&arrdep=D")

time.sleep(5) # Waiting unitl the data is loaded

soup = BeautifulSoup(driver.page_source, "html.parser")

theader = soup.select_one("thead.flight-table__header")

head = []
for row in theader.select("tr"):
    cols = row.find_all("th")
    head = [col.text.strip() for col in cols]

value = []
tbody = soup.select_one("tbody.flight-table__body")
for row in tbody.select("tr.flight-table__row"):
    cols = row.find_all("td")

    row_values = []

    for col in cols:
        text = col.text.strip()
        img = col.find("img")
        if img:
            img_src = 'https://www.ind.com'+img.get("src")
            combined = f"{text} ({img_src})" if text else img_src
        else:
            combined = text
        row_values.append(combined)

    value.append(row_values)

driver.quit()

In [13]:
import pandas as pd
df_d = pd.DataFrame(value, columns=head)
df_d

,Airline,Flight #,Destination,ETA,Gate #,Status
0,https://www.ind.com/assets/images/airlines/wn.png,WN 117,Orlando Int'l,5:00 am,B21,Departed
1,https://www.ind.com/assets/images/airlines/wn.png,WN 330,Dallas/Love Field,5:20 am,B24,Departed
2,https://www.ind.com/assets/images/airlines/aa.png,AA 2782,Dallas-Ft. Worth,5:23 am,B9,Departed
3,https://www.ind.com/assets/images/airlines/wn.png,WN 397,Baltimore,5:30 am,B20,Departed
4,https://www.ind.com/assets/images/airlines/nk.png,NK 423,Orlando Int'l,5:35 am,A20,Departed
...,...,...,...,...,...,...
147,https://www.ind.com/assets/images/airlines/nk.png,NK 425,Las Vegas,8:16 pm,A20,Delayed
148,https://www.ind.com/assets/images/airlines/wn.png,WN 1281,Phoenix,8:00 pm,B24,
149,https://www.ind.com/assets/images/airlines/aa.png,AA 1945,Dallas-Ft. Worth,8:15 pm,,On Time
150,https://www.ind.com/assets/images/airlines/wn.png,WN 4005,Orlando Int'l,8:40 pm,B23,


In [14]:
# Preprocessing
from datetime import datetime
import numpy as np

status_mapping = {
    'Progressing': 'PRG',
    'Boarding': 'BRD',
    'Departed': 'DPT',
    'Arrived': 'ARR',
    'Delayed': 'DLT',
    'Cancelled': 'CNL',
    'On Time': 'ONT',
    'Early': 'ERL'
}

df_d['FlightNumber'] = df_d['Flight #'].apply(lambda x: x.split(' ')[1])
df_d['AirlineCode'] = df_d['Flight #'].apply(lambda x: x.split(' ')[0])
df_d['AirportCode'] = 'IND' # Indianapolis
df_d['ScheduledDate'] = datetime.today().strftime('%Y%m%d')
df_d['ScheduledTime'] = df_d['ETA'].apply(
    lambda x: datetime.strptime(x.strip().lower(), "%I:%M %p").strftime("%H%M"))
df_d['EstimatedDate'] = datetime.today().strftime('%Y%m%d')
df_d['EstimatedTime'] = df_d['ETA'].apply(
    lambda x: datetime.strptime(x.strip().lower(), "%I:%M %p").strftime("%H%M"))
df_d['OriginDestAirport'] = np.random.choice(df_airports['AirportCode'].dropna().unique(), size=len(df_d)) # Arbitrary data as airports data is not perfectly matching with those of df

#df['CheckinCounter'] = 
df_d['BoardingGate'] = df_d['Gate #']
#df['BaggageClaimBelt'] = 
df_d['Remarks'] = df_d['Status'].map(status_mapping)

df_d['FlightType'] = 'D'

df_d

,Airline,Flight #,Destination,ETA,Gate #,Status,FlightNumber,AirlineCode,AirportCode,ScheduledDate,ScheduledTime,EstimatedDate,EstimatedTime,OriginDestAirport,BoardingGate,Remarks,FlightType
0,https://www.ind.com/assets/images/airlines/wn.png,WN 117,Orlando Int'l,5:00 am,B21,Departed,117,WN,IND,20250502,0500,20250502,0500,PUJ,B21,DPT,D
1,https://www.ind.com/assets/images/airlines/wn.png,WN 330,Dallas/Love Field,5:20 am,B24,Departed,330,WN,IND,20250502,0520,20250502,0520,IND,B24,DPT,D
2,https://www.ind.com/assets/images/airlines/aa.png,AA 2782,Dallas-Ft. Worth,5:23 am,B9,Departed,2782,AA,IND,20250502,0523,20250502,0523,MYR,B9,DPT,D
3,https://www.ind.com/assets/images/airlines/wn.png,WN 397,Baltimore,5:30 am,B20,Departed,397,WN,IND,20250502,0530,20250502,0530,SEA,B20,DPT,D
4,https://www.ind.com/assets/images/airlines/nk.png,NK 423,Orlando Int'l,5:35 am,A20,Departed,423,NK,IND,20250502,0535,20250502,0535,PGD,A20,DPT,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,https://www.ind.com/assets/images/airlines/nk.png,NK 425,Las Vegas,8:16 pm,A20,Delayed,425,NK,IND,20250502,2016,20250502,2016,PIE,A20,DLT,D
148,https://www.ind.com/assets/images/airlines/wn.png,WN 1281,Phoenix,8:00 pm,B24,,1281,WN,IND,20250502,2000,20250502,2000,LGA,B24,NaN,D
149,https://www.ind.com/assets/images/airlines/aa.png,AA 1945,Dallas-Ft. Worth,8:15 pm,,On Time,1945,AA,IND,20250502,2015,20250502,2015,CLT,,ONT,D
150,https://www.ind.com/assets/images/airlines/wn.png,WN 4005,Orlando Int'l,8:40 pm,B23,,4005,WN,IND,20250502,2040,20250502,2040,EYW,B23,NaN,D


In [15]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('FIDS.db')  
cursor = conn.cursor()

for _, row in df_a.iterrows():
    row = row.where(pd.notnull(row), None)
    
    sql = """
    INSERT OR IGNORE INTO ActiveFlightSchedules (
        FlightNumber, AirportCode, AirlineCode,
        ScheduledDate, ScheduledTime,
        EstimatedDate, EstimatedTime,
        OriginDestAirport, Remarks, FlightType
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    
    values = (
        row['FlightNumber'],
        row['AirportCode'],
        row['AirlineCode'],
        row['ScheduledDate'],
        row['ScheduledTime'],
        row['EstimatedDate'],
        row['EstimatedTime'],
        row['OriginDestAirport'],
        row['Remarks'],
        row['FlightType']
    )
    
    cursor.execute(sql, values)
    

for _, row in df_d.iterrows():
    row = row.where(pd.notnull(row), None)
    
    sql = """
    INSERT OR IGNORE INTO ActiveFlightSchedules (
        FlightNumber, AirportCode, AirlineCode,
        ScheduledDate, ScheduledTime,
        EstimatedDate, EstimatedTime,
        OriginDestAirport, Remarks, FlightType
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    
    values = (
        row['FlightNumber'],
        row['AirportCode'],
        row['AirlineCode'],
        row['ScheduledDate'],
        row['ScheduledTime'],
        row['EstimatedDate'],
        row['EstimatedTime'],
        row['OriginDestAirport'],
        row['Remarks'],
        row['FlightType']
    )
    
    cursor.execute(sql, values)
conn.commit()
conn.close()
